<a href="https://colab.research.google.com/github/Shashank1202/Deep_Learning/blob/main/Computer%20Vision/PyTorch_CNN_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [10]:
class CNN(nn.Module):
    def __init__(self, in_channels=1, num_classes=10):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(
            in_channels=in_channels,
            out_channels=8,
            kernel_size=3,
            stride=1,
            padding=1,
        )
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(
            in_channels=8,
            out_channels=16,
            kernel_size=3,
            stride=1,
            padding=1,
        )
        self.fc1 = nn.Linear(16 * 7 * 7, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)
        return x

In [6]:
device= torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
in_channel= 1
num_classes= 10
learning_rate= 0.001
batch_size= 64
num_epochs= 5

In [8]:
train_dataset= datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download= True )

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:02<00:00, 3910201.97it/s]


Extracting dataset/MNIST/raw/train-images-idx3-ubyte.gz to dataset/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 482685.40it/s]


Extracting dataset/MNIST/raw/train-labels-idx1-ubyte.gz to dataset/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 3794771.52it/s]


Extracting dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to dataset/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 4201704.62it/s]

Extracting dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to dataset/MNIST/raw



In [9]:
train_loader= DataLoader(dataset=train_dataset, batch_size= batch_size, shuffle= True)
test_dataset= datasets.MNIST(root='dataset/', train=False, transform= transforms.ToTensor(), download=True)
test_loader= DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle= True)

In [11]:
model= CNN().to(device)

In [12]:
criterion= nn.CrossEntropyLoss()
optimizer= optim.Adam(model.parameters(), lr= learning_rate)

In [13]:
for epoch in range(num_epochs):
    for batch_idx,(data, targets) in enumerate(train_loader):
        data= data.to(device=device)
        targets= targets.to(device=device)

        #forward
        scores= model(data)
        loss= criterion(scores, targets)

        #backward
        optimizer.zero_grad()
        loss.backward()

        #gradient descent or adam step
        optimizer.step()

In [15]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Checking the accuracy on training data")
    else:
        print("Checking the accuracy on Test data")

    num_correct= 0
    num_samples= 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x= x.to(device=device)
            y= y.to(device=device)

            scores= model(x)
            _, predictions= scores.max(1)
            num_correct += (predictions==y).sum()
            num_samples += predictions.size(0)

        print(f'Got {num_correct}/{num_samples} with accuracy {float(num_correct)/float(num_samples)*100:2f}')

    model.train()

check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking the accuracy on training data
Got 59094/60000 with accuracy 98.490000
Checking the accuracy on Test data
Got 9849/10000 with accuracy 98.490000
